In [1]:
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
import itertools

In [2]:
label_mapping = {
    'negative': 1,
    'Negative': 1,
    'neutral': 2,
    'Neutral': 2,
    'positive': 3,
    'Positive': 3
}

# 1. Iteracja

In [3]:
files_it1 = {
    'Bartek': 'annotations/texts/it1/bartek.csv',
    'Darek': 'annotations/texts/it1/darek.csv',
    'Patryk': 'annotations/texts/it1/patryk.csv'
}

data = {}
for name, filepath in files_it1.items():
    try:
        df = pd.read_csv(filepath)
        print(f"Plik '{filepath}' został pomyślnie wczytany.")
        
        df['label'] = df['label'].map(label_mapping)
        
        if df['label'].isnull().any():
            unknown_labels = df[df['label'].isnull()]['label']
            print(f"Uwaga: W pliku '{filepath}' znaleziono nieznane etykiety: {unknown_labels.unique()}")
            exit(1)
        
        data[name] = df
    except Exception as e:
        print(f"Nie udało się wczytać pliku '{filepath}': {e}")
        exit(1)

row_counts = {name: df.shape[0] for name, df in data.items()}
if len(set(row_counts.values())) != 1:
    print("Pliki mają różną liczbę wierszy:")
    for name, count in row_counts.items():
        print(f" - {name}: {count} wierszy")
    exit(1)
else:
    print("\nWszystkie pliki mają taką samą liczbę wierszy.")

num_rows = data['Bartek'].shape[0]
order_match = True
for i in range(num_rows):
    texts = [data[name].iloc[i]['text'] for name in files_it1.keys()]
    if not all(text == texts[0] for text in texts[1:]):
        print(f"Różnica w wierszu {i+1}.")
        order_match = False
        break

if order_match:
    print("Kolejność wierszy jest zgodna we wszystkich plikach.")
else:
    print("Kolejność wierszy różni się między plikami. Sprawdź dane wejściowe.")
    exit(1)

Plik 'annotations/texts/it1/bartek.csv' został pomyślnie wczytany.
Plik 'annotations/texts/it1/darek.csv' został pomyślnie wczytany.
Plik 'annotations/texts/it1/patryk.csv' został pomyślnie wczytany.

Wszystkie pliki mają taką samą liczbę wierszy.
Kolejność wierszy jest zgodna we wszystkich plikach.


In [4]:
combined = pd.DataFrame()
combined['text'] = data['Bartek']['text']
combined['Bartek'] = data['Bartek']['label']
combined['Darek'] = data['Darek']['label']
combined['Patryk'] = data['Patryk']['label']

print("Przykładowe dane po połączeniu etykiet:")
combined.head()

Przykładowe dane po połączeniu etykiet:


,text,Bartek,Darek,Patryk
0,"Lakier roweru bardzo kiepskiej jakości , robią...",1,1,1
1,Nie jestem zadowolony z zakupu . Przede wszyst...,1,1,1
2,Szukając klucze tej wielkości brał em dwa pod ...,1,3,2
3,Długo szukała m odpowiedniego fotelika dla bar...,1,1,2
4,Kupił em pociąg ( w zestawie kilka torów prost...,1,1,1


In [5]:
def percent_agreement(series1, series2):
    return (series1 == series2).mean()

annotators = list(files_it1.keys())
kappa_results = {}
print("Współczynniki Kappa Cohena dla każdej pary annotatorów:")
for i in range(len(annotators)):
    for j in range(i+1, len(annotators)):
        annot1 = annotators[i]
        annot2 = annotators[j]
        kappa = cohen_kappa_score(combined[annot1], combined[annot2])
        agreement = percent_agreement(combined[annot1], combined[annot2]) * 100
        kappa_results[(annot1, annot2)] = kappa
        print(f" - {annot1} vs {annot2}:")
        print(f"   * Kappa Cohena: {kappa:.4f}")
        print(f"   * Procentowa zgodność: {agreement:.2f}%")

categories = sorted(combined[['Bartek', 'Darek', 'Patryk']].stack().unique())

Współczynniki Kappa Cohena dla każdej pary annotatorów:
 - Bartek vs Darek:
   * Kappa Cohena: 0.6554
   * Procentowa zgodność: 85.42%
 - Bartek vs Patryk:
   * Kappa Cohena: 0.6575
   * Procentowa zgodność: 85.42%
 - Darek vs Patryk:
   * Kappa Cohena: 0.8514
   * Procentowa zgodność: 93.75%


In [6]:
def create_fleiss_matrix(df, categories):
    fleiss_matrix = []
    for index, row in df.iterrows():
        counts = [row[['Bartek', 'Darek', 'Patryk']].tolist().count(category) for category in categories]
        fleiss_matrix.append(counts)
    return np.array(fleiss_matrix)

fleiss_data = create_fleiss_matrix(combined, categories)

kappa_fleiss = fleiss_kappa(fleiss_data, method='fleiss')
print(f"Współczynnik Kappa Fleissa dla wszystkich annotatorów: {kappa_fleiss:.4f}")

Współczynnik Kappa Fleissa dla wszystkich annotatorów: 0.7195


In [7]:
percent_agreements = []
for i in range(len(annotators)):
    for j in range(i+1, len(annotators)):
        annot1 = annotators[i]
        annot2 = annotators[j]
        agreement = percent_agreement(combined[annot1], combined[annot2]) * 100
        percent_agreements.append(agreement)
average_agreement = np.mean(percent_agreements)
print(f"Średnia procentowa zgodność między annotatorami: {average_agreement:.2f}%")

Średnia procentowa zgodność między annotatorami: 88.19%


In [8]:
print("Krzyżowe tabele kontyngencji dla każdej pary annotatorów:")
for annot1, annot2 in itertools.combinations(annotators, 2):
    contingency = pd.crosstab(combined[annot1], combined[annot2])
    print(f"\n{annot1} vs {annot2}:")
    print(contingency)

Krzyżowe tabele kontyngencji dla każdej pary annotatorów:

Bartek vs Darek:
Darek    1  2  3
Bartek          
1       34  0  1
2        2  5  4
3        0  0  2

Bartek vs Patryk:
Patryk   1  2  3
Bartek          
1       33  2  0
2        2  6  3
3        0  0  2

Darek vs Patryk:
Patryk   1  2  3
Darek           
1       35  1  0
2        0  5  0
3        0  2  5


In [9]:
def interpret_kappa(kappa):
    if kappa < 0:
        return "Zgoda gorsza niż przypadkowa."
    elif kappa == 0:
        return "Brak zgodności lepszy niż przypadkowy."
    elif kappa <= 0.20:
        return "Znikoma zgodność."
    elif kappa <= 0.40:
        return "Słaba zgodność."
    elif kappa <= 0.60:
        return "Umiarkowana zgodność."
    elif kappa <= 0.80:
        return "Dobra zgodność."
    else:
        return "Bardzo dobra zgodność."

print("Interpretacja współczynników Kappa Cohena:")
for pair, kappa in kappa_results.items():
    interpretation = interpret_kappa(kappa)
    print(f" - {pair[0]} vs {pair[1]}: Kappa = {kappa:.4f} -> {interpretation}")

print(f"\nInterpretacja współczynnika Kappa Fleissa: Kappa = {kappa_fleiss:.4f} -> {interpret_kappa(kappa_fleiss)}")

Interpretacja współczynników Kappa Cohena:
 - Bartek vs Darek: Kappa = 0.6554 -> Dobra zgodność.
 - Bartek vs Patryk: Kappa = 0.6575 -> Dobra zgodność.
 - Darek vs Patryk: Kappa = 0.8514 -> Bardzo dobra zgodność.

Interpretacja współczynnika Kappa Fleissa: Kappa = 0.7195 -> Dobra zgodność.


In [10]:
def row_agreement(row):
    unique_labels = row.nunique()
    return unique_labels

combined['agreement_level'] = combined[annotators].apply(row_agreement, axis=1)

max_unique = combined['agreement_level'].max()
min_agreement_rows = combined[combined['agreement_level'] == max_unique]

print(f"\nLiczba unikalnych etykiet w wierszu wskazuje na zgodność:")
print(f" - Maksymalna liczba unikalnych etykiet: {max_unique}")
print(f" - Liczba wierszy z najmniejszą zgodnością: {min_agreement_rows.shape[0]}")

print("\nPrzykłady wierszy z najmniejszą zgodnością:")
print(min_agreement_rows[['text'] + annotators].head(10))  # Wyświetli pierwsze 10 takich wierszy


Liczba unikalnych etykiet w wierszu wskazuje na zgodność:
 - Maksymalna liczba unikalnych etykiet: 3
 - Liczba wierszy z najmniejszą zgodnością: 1

Przykłady wierszy z najmniejszą zgodnością:
                                                text  Bartek  Darek  Patryk
2  Szukając klucze tej wielkości brał em dwa pod ...       1      3       2


In [11]:
combined.to_csv('annotations/texts/it1/combined.csv', index=False)

# Wytyczne do anotacji sentymentu

1. Kategorie sentymentu:
- `1` - Negatywny, gdy większa częsc wypowiedzi jest negatywna
- `2` - Neutralny, gdy stona negatywna i pozytywna są równie liczne
- `3` - Pozytywny, gdy większość recenzji jest pozytywna

2. Kryteria anotacji

- Określenie dominującego sentymentu w tekście:
Jesli recenzja zawiera podobną ilosc wad i zalet to jest neutralna

- Kontekst i cel opinii:
Dobrze jest zdefiniować cel w jakim opinia została napisana.

- Język i ton wypowiedzi:
Jeśli recenzja jest mocno nacechowana emocjonalnie, najczesciej zbiega ku negatywnej lub pozytywnej.

- Konsekwencja w ocenianiu:
Ważne jest żeby trzymać się pewnych zasad podczas oceny wszystkich tekstów.

3. Przykłady

| Tekst | Anotacja |
|-------|----------|
| „Lakier roweru bardzo kiepskiej jakości... ODRADZAM ZAKUP” | **1 - Negatywny** |
| „Obiektyw ten przeznaczony jest dla osób które dopiero zaczynają fotografowanie... Polecam go głównie początkującym użytkownikom.” | **3 - Pozytywny** |
| „Pilot funkcjonalnie pierwsza klasa... Niestety sama jakość wykonania pozostawia wiele do życzenia.” | **2 - Neutralny** |
| „Najpierw + dostajemy produkt z najwyższej półki... Super wygląd.” | **3 - Pozytywny** |
| „Lodówka fajnie się prezentuje... ale popsuła się po 2.5 roku.” | **2 - Neutralny** |
| „Długo szukała m odpowiedniego fotelika... Poza tym fotelik wyglądał solidnie i myślę, że świetnie nadawał by się jako fotelik następny po nosidełku.” | **2 - Neutralny** |


# 2. Iteracja

In [12]:
files_it2 = {
    'Bartek': 'annotations/texts/it2/bartek.csv',
    'Darek': 'annotations/texts/it2/darek.csv',
    'Patryk': 'annotations/texts/it2/patryk.csv'
}

In [13]:
data = {}
for name, filepath in files_it2.items():
    try:
        df = pd.read_csv(filepath)
        print(f"Plik '{filepath}' został pomyślnie wczytany.")
        
        df['label'] = df['label'].map(label_mapping)
        
        if df['label'].isnull().any():
            unknown_labels = df[df['label'].isnull()]['label']
            print(f"Uwaga: W pliku '{filepath}' znaleziono nieznane etykiety: {unknown_labels.unique()}")
            exit(1)
        
        data[name] = df
    except Exception as e:
        print(f"Nie udało się wczytać pliku '{filepath}': {e}")
        exit(1)

row_counts = {name: df.shape[0] for name, df in data.items()}
if len(set(row_counts.values())) != 1:
    print("Pliki mają różną liczbę wierszy:")
    for name, count in row_counts.items():
        print(f" - {name}: {count} wierszy")
    exit(1)
else:
    print("\nWszystkie pliki mają taką samą liczbę wierszy.")

num_rows = data['Bartek'].shape[0]
order_match = True
for i in range(num_rows):
    texts = [data[name].iloc[i]['text'] for name in files_it2.keys()]
    if not all(text == texts[0] for text in texts[1:]):
        print(f"Różnica w wierszu {i+1}.")
        order_match = False
        break

if order_match:
    print("Kolejność wierszy jest zgodna we wszystkich plikach.")
else:
    print("Kolejność wierszy różni się między plikami. Sprawdź dane wejściowe.")
    exit(1)

Plik 'annotations/texts/it2/bartek.csv' został pomyślnie wczytany.
Plik 'annotations/texts/it2/darek.csv' został pomyślnie wczytany.
Plik 'annotations/texts/it2/patryk.csv' został pomyślnie wczytany.

Wszystkie pliki mają taką samą liczbę wierszy.
Kolejność wierszy jest zgodna we wszystkich plikach.


In [14]:
combined = pd.DataFrame()
combined['text'] = data['Bartek']['text']
combined['Bartek'] = data['Bartek']['label']
combined['Darek'] = data['Darek']['label']
combined['Patryk'] = data['Patryk']['label']

print("Przykładowe dane po połączeniu etykiet:")
combined.head()

Przykładowe dane po połączeniu etykiet:


,text,Bartek,Darek,Patryk
0,"Lakier roweru bardzo kiepskiej jakości , robią...",1,1,1
1,Nie jestem zadowolony z zakupu . Przede wszyst...,1,1,1
2,Szukając klucze tej wielkości brał em dwa pod ...,2,2,2
3,Długo szukała m odpowiedniego fotelika dla bar...,2,2,1
4,Kupił em pociąg ( w zestawie kilka torów prost...,1,1,1


In [15]:
def percent_agreement(series1, series2):
    return (series1 == series2).mean()

annotators = list(files_it2.keys())
kappa_results = {}
print("Współczynniki Kappa Cohena dla każdej pary annotatorów:")
for i in range(len(annotators)):
    for j in range(i+1, len(annotators)):
        annot1 = annotators[i]
        annot2 = annotators[j]
        kappa = cohen_kappa_score(combined[annot1], combined[annot2])
        agreement = percent_agreement(combined[annot1], combined[annot2]) * 100
        kappa_results[(annot1, annot2)] = kappa
        print(f" - {annot1} vs {annot2}:")
        print(f"   * Kappa Cohena: {kappa:.4f}")
        print(f"   * Procentowa zgodność: {agreement:.2f}%")

categories = sorted(combined[['Bartek', 'Darek', 'Patryk']].stack().unique())

Współczynniki Kappa Cohena dla każdej pary annotatorów:
 - Bartek vs Darek:
   * Kappa Cohena: 0.9572
   * Procentowa zgodność: 97.92%
 - Bartek vs Patryk:
   * Kappa Cohena: 0.9156
   * Procentowa zgodność: 95.83%
 - Darek vs Patryk:
   * Kappa Cohena: 0.8715
   * Procentowa zgodność: 93.75%


In [16]:
def create_fleiss_matrix(df, categories):
    fleiss_matrix = []
    for index, row in df.iterrows():
        counts = [row[['Bartek', 'Darek', 'Patryk']].tolist().count(category) for category in categories]
        fleiss_matrix.append(counts)
    return np.array(fleiss_matrix)

fleiss_data = create_fleiss_matrix(combined, categories)

kappa_fleiss = fleiss_kappa(fleiss_data, method='fleiss')
print(f"Współczynnik Kappa Fleissa dla wszystkich annotatorów: {kappa_fleiss:.4f}")

Współczynnik Kappa Fleissa dla wszystkich annotatorów: 0.9148


In [17]:
percent_agreements = []
for i in range(len(annotators)):
    for j in range(i+1, len(annotators)):
        annot1 = annotators[i]
        annot2 = annotators[j]
        agreement = percent_agreement(combined[annot1], combined[annot2]) * 100
        percent_agreements.append(agreement)
average_agreement = np.mean(percent_agreements)
print(f"Średnia procentowa zgodność między annotatorami: {average_agreement:.2f}%")

Średnia procentowa zgodność między annotatorami: 95.83%


In [18]:
print("Krzyżowe tabele kontyngencji dla każdej pary annotatorów:")
for annot1, annot2 in itertools.combinations(annotators, 2):
    contingency = pd.crosstab(combined[annot1], combined[annot2])
    print(f"\n{annot1} vs {annot2}:")
    print(contingency)

Krzyżowe tabele kontyngencji dla każdej pary annotatorów:

Bartek vs Darek:
Darek    1   2  3
Bartek           
1       31   0  0
2        1  13  0
3        0   0  3

Bartek vs Patryk:
Patryk   1   2  3
Bartek           
1       30   1  0
2        1  13  0
3        0   0  3

Darek vs Patryk:
Patryk   1   2  3
Darek            
1       30   2  0
2        1  12  0
3        0   0  3


In [19]:
def interpret_kappa(kappa):
    if kappa < 0:
        return "Zgoda gorsza niż przypadkowa."
    elif kappa == 0:
        return "Brak zgodności lepszy niż przypadkowy."
    elif kappa <= 0.20:
        return "Znikoma zgodność."
    elif kappa <= 0.40:
        return "Słaba zgodność."
    elif kappa <= 0.60:
        return "Umiarkowana zgodność."
    elif kappa <= 0.80:
        return "Dobra zgodność."
    else:
        return "Bardzo dobra zgodność."

print("\nInterpretacja współczynników Kappa Cohena:")
for pair, kappa in kappa_results.items():
    interpretation = interpret_kappa(kappa)
    print(f" - {pair[0]} vs {pair[1]}: Kappa = {kappa:.4f} -> {interpretation}")

print(f"\nInterpretacja współczynnika Kappa Fleissa: Kappa = {kappa_fleiss:.4f} -> {interpret_kappa(kappa_fleiss)}")


Interpretacja współczynników Kappa Cohena:
 - Bartek vs Darek: Kappa = 0.9572 -> Bardzo dobra zgodność.
 - Bartek vs Patryk: Kappa = 0.9156 -> Bardzo dobra zgodność.
 - Darek vs Patryk: Kappa = 0.8715 -> Bardzo dobra zgodność.

Interpretacja współczynnika Kappa Fleissa: Kappa = 0.9148 -> Bardzo dobra zgodność.


In [20]:
def row_agreement(row):
    unique_labels = row.nunique()
    return unique_labels

combined['agreement_level'] = combined[annotators].apply(row_agreement, axis=1)

max_unique = combined['agreement_level'].max()
min_agreement_rows = combined[combined['agreement_level'] == max_unique]

print(f"\nLiczba unikalnych etykiet w wierszu wskazuje na zgodność:")
print(f" - Maksymalna liczba unikalnych etykiet: {max_unique}")
print(f" - Liczba wierszy z najmniejszą zgodnością: {min_agreement_rows.shape[0]}")

print("\nPrzykłady wierszy z najmniejszą zgodnością:")
print(min_agreement_rows[['text'] + annotators].head(10))  # Wyświetli pierwsze 10 takich wierszy


Liczba unikalnych etykiet w wierszu wskazuje na zgodność:
 - Maksymalna liczba unikalnych etykiet: 2
 - Liczba wierszy z najmniejszą zgodnością: 3

Przykłady wierszy z najmniejszą zgodnością:
                                                 text  Bartek  Darek  Patryk
3   Długo szukała m odpowiedniego fotelika dla bar...       2      2       1
29  Za ta cenę spodziewał em się bardziej dopracow...       1      1       2
32  Chyba spodziewała m się czegoś więcej , po kos...       2      1       2


In [21]:
combined.to_csv('annotations/texts/it2/combined.csv', index=False)

# Analiza

In [32]:

final_result = pd.read_csv('annotations/texts/it2/final_texts.csv')

color_map = {
    'Negative': 'rgba(255, 0, 0, 0.6)',  # Red
    'Positive': 'rgba(0, 255, 0, 0.6)',  # Green
    'Neutral': 'rgba(0, 0, 255, 0.6)'    # Blue
}


final_result["len_char"] = final_result["text"].apply(lambda x: len(x))
final_result["words_count"] = final_result["text"].apply(lambda x: len(x.split()))

df = final_result

fig = make_subplots(rows=2, cols=3, subplot_titles=("Text Length - Negative", "Text Length - Neutral", "Text Length - Positive",
                                                    "Word Count - Negative", "Word Count - Neutral", "Word Count - Positive"), shared_yaxes=True)

fig.add_trace(px.histogram(df[df['label'] == 'Negative'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Negative']]).data[0], row=1, col=1)
fig.add_trace(px.histogram(df[df['label'] == 'Neutral'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Neutral']]).data[0], row=1, col=2)
fig.add_trace(px.histogram(df[df['label'] == 'Positive'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Positive']]).data[0], row=1, col=3)

fig.add_trace(px.histogram(df[df['label'] == 'Negative'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Negative']]).data[0], row=2, col=1)
fig.add_trace(px.histogram(df[df['label'] == 'Neutral'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Neutral']]).data[0], row=2, col=2)
fig.add_trace(px.histogram(df[df['label'] == 'Positive'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Positive']]).data[0], row=2, col=3)

fig.update_layout(height=800, width=1200, title_text="Distribution of Text Length and Word Count by Sentiment Class")

fig.show()

# Analiza Reviews

In [33]:
df = pd.read_csv('annotations/texts/all/rev_combined.csv')

In [34]:
df["len_char"] = df["text"].apply(lambda x: len(x))
df["words_count"] = df["text"].apply(lambda x: len(x.split()))

fig = make_subplots(rows=2, cols=3, subplot_titles=("Text Length - Negative", "Text Length - Neutral", "Text Length - Positive",
                                                    "Word Count - Negative", "Word Count - Neutral", "Word Count - Positive"), shared_yaxes=True)

fig.add_trace(px.histogram(df[df['label'] == 'Negative'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Negative']]).data[0], row=1, col=1)
fig.add_trace(px.histogram(df[df['label'] == 'Neutral'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Neutral']]).data[0], row=1, col=2)
fig.add_trace(px.histogram(df[df['label'] == 'Positive'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Positive']]).data[0], row=1, col=3)

fig.add_trace(px.histogram(df[df['label'] == 'Negative'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Negative']]).data[0], row=2, col=1)
fig.add_trace(px.histogram(df[df['label'] == 'Neutral'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Neutral']]).data[0], row=2, col=2)
fig.add_trace(px.histogram(df[df['label'] == 'Positive'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Positive']]).data[0], row=2, col=3)

fig.update_layout(height=800, width=1200, title_text="Distribution of Text Length and Word Count by Sentiment Class")

fig.show()

In [35]:


df = pd.read_csv('annotations/texts/all_texts.csv')


In [36]:
df["len_char"] = df["text"].apply(lambda x: len(x))
df["words_count"] = df["text"].apply(lambda x: len(x.split()))

fig = make_subplots(rows=2, cols=3, subplot_titles=("Text Length - Negative", "Text Length - Neutral", "Text Length - Positive", "Word Count - Negative", "Word Count - Neutral", "Word Count - Positive"), shared_yaxes=True)

fig.add_trace(px.histogram(df[df['label'] == 'Negative'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Negative']]).data[0], row=1, col=1)
fig.add_trace(px.histogram(df[df['label'] == 'Neutral'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Neutral']]).data[0], row=1, col=2)
fig.add_trace(px.histogram(df[df['label'] == 'Positive'], x="len_char", nbins=300, color_discrete_sequence=[color_map['Positive']]).data[0], row=1, col=3)

fig.add_trace(px.histogram(df[df['label'] == 'Negative'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Negative']]).data[0], row=2, col=1)
fig.add_trace(px.histogram(df[df['label'] == 'Neutral'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Neutral']]).data[0], row=2, col=2)
fig.add_trace(px.histogram(df[df['label'] == 'Positive'], x="words_count", nbins=300, color_discrete_sequence=[color_map['Positive']]).data[0], row=2, col=3)

fig.update_layout(height=800, width=1200, title_text="Distribution of Text Length and Word Count by Sentiment Class")

fig.show()